In [7]:
# %pip install langchain
# %pip install -U langchain-community
# %pip install pymupdf
# %pip install sentence-transformers
# %pip install --upgrade pip
# %pip install chromadb
%pip install llama-cpp-python

     ---------------------------------------- 0.0/50.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/50.3 MB 1.3 MB/s eta 0:00:40
     ---------------------------------------- 0.0/50.3 MB 1.3 MB/s eta 0:00:40
     --------------------------------------- 0.1/50.3 MB 657.6 kB/s eta 0:01:17
     --------------------------------------- 0.1/50.3 MB 726.2 kB/s eta 0:01:10
     --------------------------------------- 0.2/50.3 MB 908.0 kB/s eta 0:00:56
     ---------------------------------------- 0.3/50.3 MB 1.1 MB/s eta 0:00:46
     ---------------------------------------- 0.4/50.3 MB 1.4 MB/s eta 0:00:36
     ---------------------------------------- 0.6/50.3 MB 1.7 MB/s eta 0:00:29
      --------------------------------------- 0.8/50.3 MB 2.1 MB/s eta 0:00:24
      --------------------------------------- 1.1/50.3 MB 2.5 MB/s eta 0:00:20
     - -------------------------------------- 1.3/50.3 MB 2.8 MB/s eta 0:00:18
     - -------------------------------------- 1.6/50.3 M

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA

In [3]:
loader = PyMuPDFLoader("Virtual_characters.pdf")
PDF_data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)

In [5]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downl

In [3]:
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain_community.llms import LlamaCpp

# llm = LlamaCpp(
#     model_path="llama-2_q4.gguf",
#     n_gpu_layers=100,
#     n_batch=512,
#     n_ctx=2048,
#     f16_kv=True,
#     callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
#     verbose=True,
# )
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key='None', openai_api_base='http://127.0.0.1:8080/v1')

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ImportError: Could not import openai python package. Please install it with `pip install openai`.

In [6]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> 
    You are a helpful assistant eager to assist with providing better Google search results.
    <</SYS>> 
    
    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative, \
            relevant, and concise:
            {question} 
    [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a helpful assistant eager to assist with providing better Google search results. \
        Provide an answer to the following question in about 150 words. Ensure that the answer is informative, \
        relevant, and concise: \
        {question}""",
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['question'], template='<<SYS>> \n    You are a helpful assistant eager to assist with providing better Google search results.\n    <</SYS>> \n    \n    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative,             relevant, and concise:\n            {question} \n    [/INST]')

In [7]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What is Taiwan known for?"
llm_chain.invoke({"question": question})

  Taiwan is known for its vibrant culture, rich history, and stunning natural beauty. Some of its notable attractions include the bustling Night Markets, where visitors can sample local street food and buy unique souvenirs; the ancient city of Lukang, with its well-preserved traditional architecture; and the beautiful Taroko National Park, featuring marble cliffs, waterfalls, and hiking trails. Taiwan is also famous for its delicious cuisine, including dishes like beef noodle soup, oyster omelets, and bubble tea. Additionally, Taiwan has a thriving tech industry and is home to many world-renowned brands, making it a hub of innovation and technology.


llama_print_timings:        load time =    5102.15 ms
llama_print_timings:      sample time =      49.11 ms /   159 runs   (    0.31 ms per token,  3237.96 tokens per second)
llama_print_timings: prompt eval time =    5102.02 ms /    84 tokens (   60.74 ms per token,    16.46 tokens per second)
llama_print_timings:        eval time =   22931.21 ms /   158 runs   (  145.13 ms per token,     6.89 tokens per second)
llama_print_timings:       total time =   28534.12 ms /   242 tokens


{'question': 'What is Taiwan known for?',
 'text': '  Taiwan is known for its vibrant culture, rich history, and stunning natural beauty. Some of its notable attractions include the bustling Night Markets, where visitors can sample local street food and buy unique souvenirs; the ancient city of Lukang, with its well-preserved traditional architecture; and the beautiful Taroko National Park, featuring marble cliffs, waterfalls, and hiking trails. Taiwan is also famous for its delicious cuisine, including dishes like beef noodle soup, oyster omelets, and bubble tea. Additionally, Taiwan has a thriving tech industry and is home to many world-renowned brands, making it a hub of innovation and technology.'}

In [8]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [10]:
query = "Tell me about Alison Hawk's career and age"
qa.invoke(query)



> Entering new RetrievalQA chain...
 Based

Llama.generate: prefix-match hit


 on the given context, Alison Hawk is a 28-year-old female researcher.
> Finished chain.



llama_print_timings:        load time =    5102.15 ms
llama_print_timings:      sample time =       6.77 ms /    24 runs   (    0.28 ms per token,  3543.48 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    3174.29 ms /    24 runs   (  132.26 ms per token,     7.56 tokens per second)
llama_print_timings:       total time =    3245.13 ms /    25 tokens


{'query': "Tell me about Alison Hawk's career and age",
 'result': ' Based on the given context, Alison Hawk is a 28-year-old female researcher.'}